# DASHBOARD GRAPHICS

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go

In [69]:
driver = 'postgres'
username = 'ds4a'
password = 'ds4A-7eaM79'
host = 'ds4a.cm1dcdf7pnnv.us-east-2.rds.amazonaws.com'
port = 5432
database = 'team_79'

connection_string = f'{driver}://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string, encoding='utf-8')

In [111]:
TABLES_LIST=['hurto_automotores', 'hurto_personas','violencia_intrafamiliar',
             'del_sexuales','lesiones_personales','homicidios','hurto_motocicletas']
TABLES_LIST_NICK=['hurto automotores', 'hurto Personas','violencia intrafamiliar',
             'delitos sexuales','lesiones personales','homicidios','hurto motocicletas']
COLUMN_LIST=['fecha','departamento','hora']

TITLES=['Car theft in Colombia','Shoplifting in Colombia','Domestic violence in Colombia',
        'Sexual crimes in Colombia', 'Personal injuries in Colombia','Murders in Colombia',
        'Theft of motorcycles in Colombia']
LABELS=['Car theft','Shoplifting','Domestic violence','Sexual crimes', 
        'Personal injuries','Murders','Theft of motorcycles']
GEN_GRAPH_TITLE=['Crimes in Colombia By Year','Crimes in Colombia By Hour']


In [65]:
## query Caro para graph 1
one_que=[]
for idx,i in enumerate(TABLES_LIST):
    a='''SELECT extract(year FROM {col0}) AS año,
            COUNT({col1}) AS cantidad,
            '{tabn}' AS crimen
        FROM {tab}
        GROUP BY año'''.format(tab=i,tabn=TABLES_LIST_NICK[idx],col0=COLUMN_LIST[0],col1=COLUMN_LIST[1])
    one_que.append(a)

In [66]:
## query por horas
one_que2=[]
for idx,i in enumerate(TABLES_LIST):
    if i == 'del_sexuales' or i == 'lesiones_personales' or i == 'homicidios':
        pass
    else:
        a='''SELECT {col0} AS hora,
                COUNT({col1}) AS cantidad,
                '{tabn}' AS crimen
            FROM {tab}
            GROUP BY hora'''.format(tab=i,tabn=TABLES_LIST_NICK[idx],col0=COLUMN_LIST[2],col1=COLUMN_LIST[1])
    one_que2.append(a)

In [67]:
#df graph 1
df = pd.concat([pd.read_sql_query(q, engine.connect()) for q in one_que])\
    .reset_index(drop=True).sort_values('año')

In [70]:
#df graph 2
## faltan columnas de hora del_sexuales', 'lesiones_personales','homicidios'
df2 = pd.concat([pd.read_sql_query(q, engine.connect()) for q in one_que2])\
    .reset_index(drop=True).sort_values('hora')

In [109]:
## graph one, by year

fig = go.Figure()

for idx,i in enumerate(TABLES_LIST_NICK):
    fig.add_trace(
        go.Scatter(x=list(df[df.crimen==i].año),
                   y=list(df[df.crimen==i].cantidad),
                   name=LABELS[idx],
                   line=dict(color=px.colors.qualitative.Plotly[idx])))

btns=[]

## add general title
btns.append(dict(label="All",
            method="update",
            args=[{"visible": [True]*len(LABELS)},
                  {"title": GEN_GRAPH_TITLE[0]}]))

# add titles

for idx, j in enumerate(TITLES):
    visible=[False]*len(LABELS)
    visible[idx]=True
    btns.append(dict(label=LABELS[idx],
                     method="update",
                     args=[{"visible": visible},
                           {"title": j}]))


fig.update_layout(updatemenus=[dict(active=0,buttons=btns)])
fig.update_layout(title_text=GEN_GRAPH_TITLE[0])

In [133]:
## graph one, by hour

fig = go.Figure()

for idx,i in enumerate(TABLES_LIST_NICK):
        if False: # i == 'delitos sexuales' or i == 'lesiones personales' or i == 'homicidios':
            pass
        else:
            fig.add_trace(
                go.Scatter(x=list(df2[df2.crimen==i].hora),
                           y=list(df2[df2.crimen==i].cantidad),
                           name=LABELS[idx],
                           line=dict(color=px.colors.qualitative.Plotly[idx])))

btns=[]
## add general title
btns.append(dict(label="All",
            method="update",
                args=[{"visible": [True]*(len(LABELS))},
                  {"title": GEN_GRAPH_TITLE[1]}]))

# add titles
for idx, j in enumerate(TITLES):
    visible=[False]*(len(LABELS))
    visible[idx]=True
    if False: #j == 'Sexual crimes in Colombia' or j == 'Personal injuries in Colombia' or j == 'Murders in Colombia':
        pass
    else:
        btns.append(dict(label=LABELS[idx],
                         method="update",
                         args=[{"visible": visible},
                               {"title": j}]))


fig.update_layout(updatemenus=[dict(active=0,buttons=btns)])
fig.update_layout(title_text=GEN_GRAPH_TITLE[1])